In [21]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [22]:
import pandas as pd

#unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
ch_names = ['timestamps','Fp1','Fp2','C3','C4','P7','P8','O1','O2','F7','F8','F3','F4','T7','T8','P3','P4','stim','sfreq']
#unfiltered_subjects = ['Aanand_19_02.csv']
# Create an empty DataFrame to store all data
all_data = pd.DataFrame(columns=ch_names)

for file_name in unfiltered_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    all_data = pd.concat([all_data, combined_df])

C:\Users\LEGION\AppData\Local\Temp\ipykernel_13416\2273160114.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, combined_df])


In [23]:
drop1 = all_data.drop(['sfreq'], axis=1)
df = drop1.drop(['stim'], axis=1)


#drop_data = testing_data.drop(['sfreq'], axis=1)
# df2 = drop_data.drop(['stim'], axis=1)

In [24]:
df.shape

(636251, 18)

In [25]:
label_counts = df['label'].value_counts()
print(label_counts)

df1 = df

label
1.0    243767
2.0    211038
3.0    181446
Name: count, dtype: int64


In [26]:
df = df1.drop(['label'], axis=1)

In [27]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)
raw.set_eeg_reference()

raw.filter(l_freq = 1,h_freq = 30)

Creating RawArray with float64 data, n_channels=16, n_times=636251
    Range : 0 ... 636250 =      0.000 ...  5302.083 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.2s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [28]:
duration = 0.3
epochs=mne.make_fixed_length_epochs(raw, duration = duration, overlap=0,preload=True)

Not setting metadata
17673 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 17673 events and 36 original time points ...
0 bad epochs dropped


In [29]:
epochs.drop_bad()  # This will drop bad epochs
num_epochs = len(epochs)
print("Total number of epochs created:", num_epochs)

Total number of epochs created: 17673


In [30]:
epochs.get_data()

C:\Users\LEGION\AppData\Local\Temp\ipykernel_13416\1251544707.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs.get_data()


array([[[ 1.81898940e-11, -9.47492918e+04, -1.18740849e+05, ...,
         -5.23667227e+04, -5.06963990e+04, -4.86349998e+04],
        [ 3.43334250e-11, -8.71324937e+04, -1.09063184e+05, ...,
         -4.82070161e+04, -4.65285794e+04, -4.44940018e+04],
        [-1.20508048e-11, -4.18075628e+03, -3.32758466e+03, ...,
         -1.43529005e+03, -1.17122936e+03, -5.91793637e+02],
        ...,
        [ 7.27595761e-12,  1.05012965e+06,  1.34087222e+06, ...,
          5.92511837e+05,  5.75430350e+05,  5.58259093e+05],
        [ 2.27373675e-12,  2.93703433e+04,  3.14944841e+04, ...,
          1.29989777e+04,  1.23938336e+04,  1.13275035e+04],
        [ 8.73114914e-11, -5.94392622e+05, -7.53669296e+05, ...,
         -3.30316316e+05, -3.20954178e+05, -3.11955859e+05]],

       [[-4.69821077e+04, -4.59931615e+04, -4.49757397e+04, ...,
         -1.17464478e+04, -1.05092815e+04, -9.50704713e+03],
        [-4.30307319e+04, -4.22727399e+04, -4.13626010e+04, ...,
         -1.07292794e+04, -9.49589701e

In [31]:
import mne

eeg_data = epochs.get_data()

condition_ranges = {'lev1': (0, (num_epochs/3)),
                    'lev2': ((num_epochs/3) , 2*((num_epochs)/3) ),
                    'lev3': ( 2*((num_epochs)/3) , (num_epochs) )}

epochs_by_condition = {}
selected_epochs = []

for condition, (start_epoch, end_epoch) in condition_ranges.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = epochs[start_epoch:end_epoch+1].copy()
    epochs_by_condition[condition] = condition_epochs
    selected_epochs.append(condition_epochs)

selected_epochs = mne.concatenate_epochs(selected_epochs)

for condition, epoch_data in epochs_by_condition.items():
    num_epochs = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs}")

print(f"Total number of selected epochs: {len(selected_epochs)}")


C:\Users\LEGION\AppData\Local\Temp\ipykernel_13416\3628746636.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Not setting metadata
17675 matching events found
No baseline correction applied
Number of epochs for lev1: 5892
Number of epochs for lev2: 5892
Number of epochs for lev3: 5891
Total number of selected epochs: 17675


In [32]:

condition_of_interest = 'lev1'
epoch_array_for_condition = epochs_by_condition[condition_of_interest]
print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


Epoch array shape for condition lev1: (5892, 16, 36)


C:\Users\LEGION\AppData\Local\Temp\ipykernel_13416\1489624503.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


In [33]:
import pandas as pd
import numpy as np
from scipy.signal import welch



def calculate_psd_features(epoch_data, sfreq):
    psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
    psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
    psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
    psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

    freq_min = 14
    freq_max = 30

    # Find the indices corresponding to the frequency range of interest
    idx_min = np.argmax(psd_frequencies > freq_min) - 1
    idx_max = np.argmax(psd_frequencies > freq_max)

    # Extract the PSD within the specified frequency range
    psd_values_alpha_band = psd_values[:, idx_min:idx_max]
    psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
    psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
    psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
    return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band

channel_names = epochs.ch_names
sfreq = epochs.info['sfreq']  # Sampling frequency

columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
columns.append('condition')
feature_df = pd.DataFrame(columns=columns)

def append_epoch_data(epoch_array, condition_label):
    global feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=columns)
        feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

append_epoch_data(epochs_by_condition['lev1'], 'lev1')
append_epoch_data(epochs_by_condition['lev2'], 'lev2')
append_epoch_data(epochs_by_condition['lev3'], 'lev3')

print("Shape of feature DataFrame:", feature_df.shape)
print(feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 36, using nperseg = 36
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of feature DataFrame: (17675, 49)
         mean_psd_Fp1        mean_psd_Fp2         mean_psd_C3  \
0   14019.83855401552  15227.440854390266   5086.770646667451   
1   5943.651124621128   6769.638153652548   3903.022485008474   
2  3943.2044811611636   6513.917560140054  3802.7444831806806   
3     4200.6537146799   6209.002541332571   4220.187411228298   
4    3483.07390329067   4774.579480155394   2895.301011333069   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   6121.963590499551  12674.823871932515   5955.150047793241   
1  2704.9601444701434  5772.5809369075605   4455.268567353775   
2  2392.9044403164467   4210.766567579686   4208.810824736517   
3  2025.3169309636448   4639.485302082304  3908.7772728139353   
4   1497.900173455136   3311.158242358712  3290.9396270196207   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0  15136.867490550241   3986.601101419499   5585.578916320254   
1  3051.6990225830414    4207.26640227082  2404.

In [34]:
# Rename columns using a dictionary
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

# Rename columns using the rename() method
feature_df.rename(columns=new_column_names, inplace=True)

# Print the DataFrame
print(feature_df.head())


         mean_psd_Fp1        mean_psd_Fp2         mean_psd_C3  \
0   14019.83855401552  15227.440854390266   5086.770646667451   
1   5943.651124621128   6769.638153652548   3903.022485008474   
2  3943.2044811611636   6513.917560140054  3802.7444831806806   
3     4200.6537146799   6209.002541332571   4220.187411228298   
4    3483.07390329067   4774.579480155394   2895.301011333069   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   6121.963590499551  12674.823871932515   5955.150047793241   
1  2704.9601444701434  5772.5809369075605   4455.268567353775   
2  2392.9044403164467   4210.766567579686   4208.810824736517   
3  2025.3169309636448   4639.485302082304  3908.7772728139353   
4   1497.900173455136   3311.158242358712  3290.9396270196207   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0  15136.867490550241   3986.601101419499   5585.578916320254   
1  3051.6990225830414    4207.26640227082  2404.4471825128253   
2  12570.118034746289  

In [35]:
feature_df

,mean_psd_Fp1,mean_psd_Fp2,mean_psd_C3,mean_psd_C4,mean_psd_P7,mean_psd_P8,mean_psd_O1,mean_psd_O2,mean_psd_F7,mean_psd_F8,...,stddev_psd_O2,stddev_psd_F7,stddev_psd_F8,standarddev_psd_F3,standarddev_psd_F4,standarddev_psd_T7,standarddev_psd_T8,standarddev_psd_P3,standarddev_psd_P4,condition
0,14019.83855401552,15227.440854390266,5086.770646667451,6121.963590499551,12674.823871932515,5955.150047793241,15136.867490550241,3986.601101419499,5585.578916320254,10636.382517248674,...,4987.396364758903,3851.6639977222026,11994.068148089716,15423.667173323483,24184.29480432775,15006.8431942358,1005834.754283881,15799.601056146124,364847.5909689109,lev1
1,5943.651124621128,6769.638153652548,3903.022485008474,2704.9601444701434,5772.5809369075605,4455.268567353775,3051.6990225830414,4207.26640227082,2404.4471825128253,10357.865472027468,...,5643.101672613831,2297.844822342982,14066.1609485749,16080.934858146491,3716.411136759295,15148.679426569917,172828.60283299847,16464.030737409445,56253.405459093185,lev1
2,3943.2044811611636,6513.917560140054,3802.7444831806806,2392.9044403164467,4210.766567579686,4208.810824736517,12570.118034746289,3199.318900654673,3667.6210904423256,7246.938154602489,...,4494.6231774903135,4091.3360964425187,10243.61066776888,11982.915433947688,3498.1909158460103,11631.528569495806,11698.641019682987,12554.0637691322,12241.253511981029,lev1
3,4200.6537146799,6209.002541332571,4220.187411228298,2025.3169309636448,4639.485302082304,3908.7772728139353,5409.115216717494,3344.1673574566553,3212.6528690284194,5876.825588431075,...,4358.464773079821,3635.4556986103726,8064.180533783141,9771.478924231806,2815.9524653287463,9506.127431373341,2475.0006516005483,10266.108016792114,2259.8709067590135,lev1
4,3483.07390329067,4774.579480155394,2895.301011333069,1497.900173455136,3311.158242358712,3290.9396270196207,5224.376942889191,2928.5358107248553,1857.9093790335066,7027.3112752916595,...,3358.289674081003,2059.8345669862424,8677.07882328051,10798.596980855558,1208.4613940597983,10294.186638460633,1383.0116904659515,10708.89925552426,499.43239781954077,lev1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17670,245228.9751930815,12892.301553842279,43334.46870115138,69636.87234673019,13579.767164154002,9225.099629858667,16105.197495113049,27646.813377899718,63433.14480038072,5902.3218701280275,...,24400.584054416773,30122.46429752851,3432.8130739038,3531.977560920483,71052.96957061117,5595.195871497714,4558.682202865514,11172.903353301786,8785.177537783457,lev3
17671,56436.00946683041,11485.476339498697,7972.556110742608,11583.68184299625,10932.630284805879,11367.393706095196,9000.70310632132,10477.987774387993,2358.797471804464,2644.1527085502685,...,10897.666803621043,1690.508148848673,2458.3883197298483,2025.6629703997903,350830.9803308574,1062.2374817896134,743.3694412555585,1601.3503427374608,769.1067573545001,lev3
17672,14741.151005394984,4223.4798413654335,4383.178189661165,3367.0280173005794,3994.7682206437717,3969.73788363514,3933.789883645894,3440.852084969916,2670.3700290787388,445.3438498299732,...,5726.824989531533,2956.043314127499,415.48823594171466,1410.8283311819982,332882.1307625571,459.6249934963078,323.3231020251433,443.66874049562455,994.6938160276579,lev3
17673,284306.6579340331,8376.91526740655,9879.229697621051,21558.512751875034,4897.522583257876,4153.250795214725,7866.315496204395,5295.894001127902,8607.399238684347,3935.4043165092135,...,6298.774476497425,14822.25620001772,7624.4604286970225,7763.887713889802,1710853.6750096863,7315.699918036716,4687.959063917764,5084.0429332336125,16773.25744579129,lev3


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9171145685997171


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

gb_classifier.fit(X_train, y_train)

y_pred = gb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8823196605374823


In [ ]:
X_test_loaded = pd.read_csv('X_test.csv')
y_test_loaded = pd.read_csv('y_test.csv', header=None).values.flatten() 

: 

In [38]:
# def append_epoch_data(epoch_array, condition_label):
#         global feature_df
#         for epoch in epoch_array:
#             psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
#             epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
#             epoch_features = np.append(epoch_features, condition_label)
#             epoch_df = pd.DataFrame([epoch_features], columns=columns)
#             feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

# def calculate_psd_features(epoch_data, sfreq):
#         psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
#         psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
#         psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
#         psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

#         freq_min = 14
#         freq_max = 30

#         # Find the indices corresponding to the frequency range of interest
#         idx_min = np.argmax(psd_frequencies > freq_min) - 1
#         idx_max = np.argmax(psd_frequencies > freq_max)

#         # Extract the PSD within the specified frequency range
#         psd_values_alpha_band = psd_values[:, idx_min:idx_max]
#         psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
#         psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
#         psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
#         return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band


In [39]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     rf_classifier.fit(X_train, y_train)
#     y_pred = rf_classifier.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()
